In [9]:
import meb
from meb import utils
from meb import datasets
from meb import core
from meb import models

from functools import partial
from typing import List, Tuple

import numpy as np
import pandas as pd
from numba import jit, njit
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import timm
from tqdm import tqdm
import torch.nn.functional as F

pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
c = datasets.CrossDataset(resize=64, color=True, preload=True)
df = c.data_frame
data = c.data

Loading data: 100%|███████████████████████████| 860/860 [03:38<00:00,  3.94it/s]


In [10]:
#interpolate samples with less than 8 frames
n_frames = 8
for i, video in enumerate(data):
    if video.shape[0] < n_frames:
        new_shape = (n_frames,) + video.shape[1:-1]
        video = torch.tensor(video).permute(3, 0, 1, 2).unsqueeze(0).float()
        new_video = F.interpolate(video, size=new_shape, mode="trilinear")
        data[i] = new_video.squeeze(0).permute(1, 2, 3, 0).byte().numpy()

In [11]:
class Net(nn.Module):
    def __init__(self, class_num, dropout=0.5):
        super().__init__()
        self.class_num = class_num
        h1 = 32
        h2 = 64
        h3 = 256
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=h1, kernel_size=(1, 5, 5), stride=1)
        self.pool = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 3, 3))
        self.bn1 = nn.BatchNorm3d(h1)
        self.drop1 = nn.Dropout3d(dropout)
        
        self.conv2 = nn.Conv3d(in_channels=h1, out_channels=h2, kernel_size=(2, 3, 3), stride=1)
        self.bn2 = nn.BatchNorm3d(h2)
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.drop2 = nn.Dropout3d(dropout)

        self.fc1 = nn.Linear(9 ** 2 * 2 * h2, h3)
        self.fc = nn.Linear(h3, self.class_num)
        self.drop3 = nn.Dropout(dropout)

        self.alpha = nn.Parameter(torch.log(torch.tensor(10.0)))
        self.r1 = nn.Parameter(torch.log(torch.tensor(0.4)))
        self.r2 = nn.Parameter(torch.log(torch.tensor(0.05)))
        
    def forward(self, x):
        W = calculate_W(6, torch.exp(self.alpha), torch.exp(self.r1), torch.exp(self.r2))
        x = tensor_contraction(x, W)
        x = x[:, :, 1:]
     
        x = self.drop1(self.bn1(self.pool(F.relu(self.conv1(x)))))
        x = self.drop2(self.bn2(self.pool2(F.relu(self.conv2(x)))))
        #print(x.shape)
        x = x.view(x.shape[0], -1)

        x = F.relu(self.fc1(x))
        x = self.drop3(x)
        x = self.fc(x)
        return x

In [14]:
class LEDValidator(core.CrossDatasetValidator):
    def __init__(self, config: Config, verbose: bool = True):
        super().__init__(config)
        self.verbose = verbose
    def train_model(
        self,
        train_loader: torch.utils.data.DataLoader,
        test_loader: torch.utils.data.DataLoader,
    ) -> None:
        """Main training loop. Can be overriden for custom training loops."""
        for epoch in tqdm(range(self.cf.epochs), disable=self.disable_tqdm):
            if epoch == 50:
                self.optimizer.param_groups[0]["lr"] *= 0.1
            self.train_one_epoch(epoch, train_loader)
            if self.scheduler:
                self.scheduler.step(epoch + 1)
            if self.cf.validation_interval:
                if (epoch + 1) % self.cf.validation_interval == 0:
                    train_metrics = self.evaluate_model(train_loader)
                    test_metrics, outputs_test = self.evaluate_model(
                        test_loader, test=True
                    )
                    self.printer.print_train_test_validation(
                        train_metrics, test_metrics, epoch
                    )
    def setup_training(self) -> None:
        """
        Sets up the training modules, including model, criterion, optimizer, scheduler
        and mixup.
        """
        self.model = self.cf.model()
        self.criterion = self.cf.criterion()
        self.model.to(self.cf.device)
        self.optimizer = self.cf.optimizer(self.model.parameters())
        self.optimizer = self.cf.optimizer(
            [
            {"params": list(self.model.parameters())[:3], "lr": 0.1},
            {"params": list(self.model.parameters())[3:]}
            ]
        )
        self.scheduler = (
            self.cf.scheduler(self.optimizer) if self.cf.scheduler else None
        )
        self.mixup_fn = self.cf.mixup_fn() if self.cf.mixup_fn else None

In [19]:
class Config(core.Config):
    device = torch.device("cuda:1")
    action_units = utils.dataset_aus["cross"]
    epochs = 400
    batch_size = 64
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]
    train_transform = {
        "spatial": None,
        "temporal": datasets.NoisyUniformTemporalSubsample(6),
    }
    test_transform = {
        "spatial": None,
        "temporal": datasets.UniformTemporalSubsample(6),
    }
    model = partial(Net, class_num=len(action_units))

In [13]:
def calculate_W(T, alpha=20, r1=0.4, r2=0.05):
    W = torch.zeros(T, T, dtype=torch.float).to(Config.device)
    #construct W
    for i in range(T):
        for j in range(T):
            a = j - i
            b = min(1, i)
            if j > i:
                W[i, j] = alpha * (1 - r1) ** a * r1 ** b - alpha * (1 - r2) ** a * r2 ** b
            elif j == i:
                W[i, j] = alpha * (r1 - r2)
    return W

def tensor_contraction(x, W):
    out = torch.einsum("scfhw,fx->scxhw", x, W)
    div = torch.einsum("scfhw,fx->scxhw", x, torch.abs(W))
    # For stability
    out /= div + 1
    # Keep first frame as original
    out[:, :, 0] = x[:, :, 0]
    return out

In [20]:
LEDValidator(Config).validate_n_times(df, data, n_times=5)

 60%|██████████████████████▊               | 3/5 [8:36:13<5:44:12, 10326.19s/it]/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/home/tvaranka/anaconda3/envs/python3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
100%|███████████████████████████████████████| 5/5 [14:20:50<00:00, 10330.07s/it]

MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
73.2 & 69.6 & 73.7 & 51.9 & 49.6 & 54.3 & 55.2 & 53.1 & 60.6 & 64.8 & 65.9 & 64.9 & 61.4

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
67.2 & 67.3 & 58.6 & 63.7 & 60.5 & 55.4 & 62.1
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
52.7 & 45.7 & 63.7 & 7.9 & 0.7 & 19.3 & 13.6 & 8.5 & 26.5 & 36.7 & 33.0 & 31.7 & 28.3

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
37.5 & 38.6 & 22.3 & 33.9 & 26.8 & 16.7 & 29.3
